In [18]:
import pandas as pd
import numpy as np
import sqlite3

In [48]:
targetDB_path = '/data/sdecesco/databases/druggability/TargetDB_20_02_19.db'
connector = sqlite3.connect(targetDB_path)
required_columns = ['alias_symbol', 'ensembl_gene_id', 'prev_symbol', 'symbol', 'uniprot_ids']
gene_id_query = """SELECT * FROM hgnc"""
gene_xref = pd.read_sql(gene_id_query, con=connector)
xref_df = gene_xref[gene_xref.xref_name.isin(required_columns)]
xref_piv = xref_df.pivot_table(index='hgnc_id', values='xref_value', columns='xref_name',
                               aggfunc=lambda x: ';'.join(x), fill_value='')
xref_piv = xref_piv.replace('', np.nan)
xref_piv = xref_piv.dropna(subset=['uniprot_ids'])
for col in required_columns:
    if col not in xref_piv.columns:
        xref_piv[col] = ''
xref_piv.uniprot_ids = xref_piv.uniprot_ids.apply(lambda x: list(filter(None, x.split(';'))))
connector.close()
output = xref_piv.reset_index()
output.drop_duplicates(subset=['hgnc_id', 'ensembl_gene_id', 'symbol'], inplace=True)
output.index = output['hgnc_id']

output

In [52]:
gene_xref = pd.read_json('json_files/hgnc_19_02_19.zip')

In [65]:
required_columns = ['alias_symbol', 'ensembl_gene_id', 'prev_symbol', 'symbol', 'uniprot_ids']
xref_df = gene_xref[gene_xref.xref_name.isin(required_columns)]
xref_piv = xref_df.pivot_table(index='hgnc_id', values='xref_value', columns='xref_name',
                               aggfunc=lambda x: ';'.join(x), fill_value='')
xref_piv = xref_piv.replace('', np.nan)
xref_piv = xref_piv.dropna(subset=['uniprot_ids'])
xref_piv = xref_piv.replace(np.nan,'')
for col in required_columns:
    if col not in xref_piv.columns:
        xref_piv[col] = ''
xref_piv.uniprot_ids = xref_piv.uniprot_ids.apply(lambda x: list(filter(None, x.split(';'))))
connector.close()
output = xref_piv.reset_index()
output.drop_duplicates(subset=['hgnc_id', 'ensembl_gene_id', 'symbol'], inplace=True)
output.index = output['hgnc_id']

In [79]:
output = output.reset_index(drop=True)
output[output.symbol == 'RGS3']

xref_name,hgnc_id,alias_symbol,ensembl_gene_id,prev_symbol,symbol,uniprot_ids
19686,HGNC:9999,C2PA;FLJ20370;PDZ-RGS3,ENSG00000138835,,RGS3,[P49796]
